# Домашнее задание 7

Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

## Импорт

In [1]:
!pip install stop-words pymorphy2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=79035815c701fc9adf2c404d266be14a22d26fa35bdc56eb82cef4ffa14beb7f
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=623eef7f7f2fdbc93b907de69f2a8fd8d74d71957722088528e703931799a052
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built stop-words docopt


In [2]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import Counter
from functools import lru_cache
from tqdm import tqdm_notebook

Загрузка данных

In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/val.csv")

In [4]:
df_train.head(10)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1
5,5,"Манчестер через час играет, а я не дома (",0
6,6,RT @qukanacipr: да) а я в 2004 в жабу пришол и...,1
7,7,"Момент из ""Мальчик и маньяк"") Ражик гладит по ...",1
8,8,"Я просто неудачик, поцарапал экран на телефоне ((",0
9,9,хахаа тот день запомнился надолго) http://t.co...,1


In [5]:
df_test.head(10)

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."
5,204155,в Питере какой то Сайлент Хилл) http://t.co/gB...
6,204156,@Brokerrr2 ухты какой молодец)) ахаха) а ты со...
7,204157,"Мальчишки-кадеты, осталось детство где-то.))\n..."
8,204158,RT @MrFarneo: Было жутко больно :-/ http://t.c...
9,204159,Ромка что то раздобрел и купилмне киндер) Стра...


In [6]:
df_val.head(10)

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0
5,181472,RT @xenia_sobchak: Прочитала письмо Охлобыстин...,0
6,181473,@1ka2ta3 а у меня неделю нет алгебры и геометр...,1
7,181474,RT @fufepavibenu: вот пророчил я пришествие уб...,0
8,181475,франц фердинанд наконец скатился в хоровое пен...,0
9,181476,Смотрю золотой граммофон) как же круто ведут У...,1


Обработка данных

In [49]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [50]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [51]:
counts = Counter()
for sequence in text_corpus_train:
    counts.update(sequence.split())

In [52]:
print("num_words before:",len(counts.keys()))

num_words before: 256036


In [53]:
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words after: 65613


In [54]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [55]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels, w2index, used_length):
        self._txts = txts
        self._labels = labels
        self._length = used_length
        self._w2index = w2index

    def __len__(self):
        return len(self._txts)

    @lru_cache(50000)
    def encode_sentence(self, txt):
        encoded = np.zeros(self._length, dtype=int)
        enc1 = np.array([self._w2index.get(word, self._w2index["UNK"]) for word in txt.split()])
        length = min(self._length, len(enc1))
        encoded[:length] = enc1[:length]
        return encoded, length

    def __getitem__(self, index):
        encoded, length = self.encode_sentence(self._txts[index])
        return torch.from_numpy(encoded.astype(np.int32)), self._labels[index], length

In [56]:
max([len(i.split()) for i in text_corpus_train])

27

Dataset и Dataloader

In [57]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(text_corpus_train, y_train, vocab2index, 27)
valid_dataset = TwitterDataset(text_corpus_valid, y_val, vocab2index, 27)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=512,
                          shuffle=True,
                          num_workers=3)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=512,
                          shuffle=False,
                          num_workers=1)

## LSTM

In [19]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(lstm_out)

lstm_init = LSTMFixedLen(len(vocab2index), 30, 20)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [20]:
for epoch in tqdm_notebook(range(15)):
    lstm_init.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels, lengths = data[0], data[1], data[2]
        inputs = inputs.long()
        labels = labels.long().view(-1, 1)

        optimizer.zero_grad()

        outputs = lstm_init(inputs, lengths)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    lstm_init.eval()
    loss_accumed = 0
    for X, y, lengths in valid_loader:
        X = X.long()
        y = y.long().view(-1, 1)
        output = lstm_init(X, lengths)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

<ipython-input-20-3d3cce13965f>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(15)):


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 0 valid_loss 32.37789535522461
Epoch 1 valid_loss 30.645692825317383
Epoch 2 valid_loss 30.26100730895996
Epoch 3 valid_loss 29.961517333984375
Epoch 4 valid_loss 29.742958068847656
Epoch 5 valid_loss 29.56521224975586
Epoch 6 valid_loss 29.850975036621094
Epoch 7 valid_loss 30.215999603271484
Epoch 8 valid_loss 30.28109359741211
Epoch 9 valid_loss 30.16170883178711
Epoch 10 valid_loss 29.554157257080078
Epoch 11 valid_loss 29.86191177368164
Epoch 12 valid_loss 29.481422424316406
Epoch 13 valid_loss 29.566438674926758
Epoch 14 valid_loss 29.118221282958984
Training is finished!


GRU

In [60]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.gru(x)
        return self.linear(lstm_out)

gru_init = GRUFixedLen(len(vocab2index), 64, 32)
optimizer = torch.optim.Adam(gru_init.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [61]:
for epoch in tqdm_notebook(range(15)):
    gru_init.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels, lengths = data[0], data[1], data[2]
        inputs = inputs.long()
        labels = labels.long().view(-1, 1)

        optimizer.zero_grad()

        outputs = gru_init(inputs, lengths)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    gru_init.eval()
    loss_accumed = 0
    for X, y, lengths in valid_loader:
        X = X.long()
        y = y.long().view(-1, 1)
        output = gru_init(X, lengths)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

<ipython-input-61-b968ebc92a53>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(15)):


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 0 valid_loss 31.355281829833984
Epoch 1 valid_loss 30.11155128479004
Epoch 2 valid_loss 29.67523765563965
Epoch 3 valid_loss 29.485925674438477
Epoch 4 valid_loss 29.665529251098633
Epoch 5 valid_loss 29.09911346435547
Epoch 6 valid_loss 29.087541580200195
Epoch 7 valid_loss 29.109933853149414
Epoch 8 valid_loss 29.27334213256836
Epoch 9 valid_loss 29.256975173950195
Epoch 10 valid_loss 29.532241821289062
Epoch 11 valid_loss 29.87099266052246
Epoch 12 valid_loss 29.485214233398438
Epoch 13 valid_loss 29.6550235748291
Epoch 14 valid_loss 29.916019439697266
Training is finished!


Модель лстм показала себя в конечных результах лучше чем модель гру.
Но если брать результамы модели лстм где с 7 по 10 эпоху мы видим ухудшение потерь, а дальще улучшниее то в модели гру надо было делать остановку на 7-8 эпохе обучение где потерии показывали лучшее результаты
